# Google Cloud Natural Language API

- [Natural Language API Client Libraries](https://cloud.google.com/natural-language/docs/reference/libraries#client-libraries-usage-python)
- [Google Cloud Natural Language API Python Samples](https://github.com/GoogleCloudPlatform/python-docs-samples/tree/master/language/cloud-client/v1)

First, authenticate by running the following command in an interactive terminal:
```
gcloud auth application-default login
```

In [2]:
texts = """Hello, world!
President Obama is speaking at the White House.
Ladies and gentlemen!
世界、こんにちは！
今日は天気いいですね！
悲しいニュースですね
素晴らしい
Google Cloud Natural Language API は、使いやすい REST API を介して強力な機械学習モデルを提供することで、テキストの構造と意味を解析できるようにします。この API を使用すれば、ドキュメント、ニュース記事、ブログ記事に含まれる人、場所、イベントなどに関する情報を抽出できるようになります。ソーシャル メディア上のコメントから商品に対するセンチメント（感情）を把握したり、コールセンターやメッセージ アプリに寄せられた消費者の意見から顧客満足度を分析したりすることができます。リクエストでアップロードしたテキストを分析することも、Google Cloud Storage のドキュメント ストレージ上のデータを分析することもできます。"""

In [3]:
# https://github.com/GoogleCloudPlatform/python-docs-samples/blob/master/language/cloud-client/v1/snippets.py
import argparse

from google.cloud import language
import six


def sentiment_text(text):
    """Detects sentiment in the text."""
    language_client = language.Client()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    # Instantiates a plain text document.
    document = language_client.document_from_text(text)

    # Detects sentiment in the document. You can also analyze HTML with:
    #   document.doc_type == language.Document.HTML
    sentiment = document.analyze_sentiment().sentiment
    
    print('Score: {}'.format(sentiment.score))
    print('Magnitude: {}'.format(sentiment.magnitude))


def sentiment_file(gcs_uri):
    """Detects sentiment in the file located in Google Cloud Storage."""
    language_client = language.Client()

    # Instantiates a plain text document.
    document = language_client.document_from_url(gcs_uri)

    # Detects sentiment in the document. You can also analyze HTML with:
    #   document.doc_type == language.Document.HTML
    sentiment = document.analyze_sentiment().sentiment

    print('Score: {}'.format(sentiment.score))
    print('Magnitude: {}'.format(sentiment.magnitude))


def entities_text(text):
    """Detects entities in the text."""
    language_client = language.Client()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    # Instantiates a plain text document.
    document = language_client.document_from_text(text)

    # Detects entities in the document. You can also analyze HTML with:
    #   document.doc_type == language.Document.HTML
    entities = document.analyze_entities().entities

    for entity in entities:
        print('=' * 20)
        print(u'{:<16}: {}'.format('name', entity.name))
        print(u'{:<16}: {}'.format('type', entity.entity_type))
        print(u'{:<16}: {}'.format('metadata', entity.metadata))
        print(u'{:<16}: {}'.format('salience', entity.salience))
        print(u'{:<16}: {}'.format('wikipedia_url',
              entity.metadata.get('wikipedia_url', '-')))


def entities_file(gcs_uri):
    """Detects entities in the file located in Google Cloud Storage."""
    language_client = language.Client()

    # Instantiates a plain text document.
    document = language_client.document_from_url(gcs_uri)

    # Detects sentiment in the document. You can also analyze HTML with:
    #   document.doc_type == language.Document.HTML
    entities = document.analyze_entities().entities

    for entity in entities:
        print('=' * 20)
        print(u'{:<16}: {}'.format('name', entity.name))
        print(u'{:<16}: {}'.format('type', entity.entity_type))
        print(u'{:<16}: {}'.format('metadata', entity.metadata))
        print(u'{:<16}: {}'.format('salience', entity.salience))
        print(u'{:<16}: {}'.format('wikipedia_url',
              entity.metadata.get('wikipedia_url', '-')))


def syntax_text(text):
    """Detects syntax in the text."""
    language_client = language.Client()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    # Instantiates a plain text document.
    document = language_client.document_from_text(text)

    # Detects syntax in the document. You can also analyze HTML with:
    #   document.doc_type == language.Document.HTML
    tokens = document.analyze_syntax().tokens

    for token in tokens:
        print(u'{}: {}'.format(token.part_of_speech, token.text_content))

#
for text in texts.split('\n'):
    print('====================================================================================================')
    print('Text: {}'.format(text))
    sentiment_text(text)
    entities_text(text)
    print('================================================================================')
    syntax_text(text)

Text: Hello, world!
Score: 0.4
Magnitude: 0.4
name            : world
type            : LOCATION
metadata        : {}
salience        : 1
wikipedia_url   : -
<google.cloud.language.syntax.PartOfSpeech object at 0x7f2efcba4470>: Hello
<google.cloud.language.syntax.PartOfSpeech object at 0x7f2efcba4320>: ,
<google.cloud.language.syntax.PartOfSpeech object at 0x7f2efcba4c50>: world
<google.cloud.language.syntax.PartOfSpeech object at 0x7f2efcba4cc0>: !
Text: President Obama is speaking at the White House.
Score: 0.2
Magnitude: 0.2
name            : Obama
type            : PERSON
metadata        : {'mid': '/m/02mjmr', 'wikipedia_url': 'http://en.wikipedia.org/wiki/Barack_Obama'}
salience        : 0.9077594
wikipedia_url   : http://en.wikipedia.org/wiki/Barack_Obama
name            : White House
type            : LOCATION
metadata        : {'mid': '/m/081sq', 'wikipedia_url': 'http://en.wikipedia.org/wiki/White_House'}
salience        : 0.092240565
wikipedia_url   : http://en.wikipedia.org/

In [6]:
# https://github.com/GoogleCloudPlatform/python-docs-samples/blob/master/language/cloud-client/v1/snippets.py
from google.cloud import language
def syntax_file(gcs_uri):
    """Detects syntax in the file located in Google Cloud Storage."""
    language_client = language.Client()

    # Instantiates a plain text document.
    document = language_client.document_from_url(gcs_uri)

    # Detects syntax in the document. You can also analyze HTML with:
    #   document.doc_type == language.Document.HTML
    tokens = document.analyze_syntax().tokens

    for token in tokens:
        print(u'{}|{}'.format(token.text_content, token.part_of_speech))
    return tokens
  
# Specify some gcs_uri that you have read access to       
gcs_uri = 'gs://qa-nlp-1.appspot.com/messages/-KoD6Ht5pST_rdxZTXkZ'
tokens = syntax_file(gcs_uri)

ＡＩ|<google.cloud.language.syntax.PartOfSpeech object at 0x7f2efcbb12b0>
の|<google.cloud.language.syntax.PartOfSpeech object at 0x7f2efcbc1dd8>
経済|<google.cloud.language.syntax.PartOfSpeech object at 0x7f2efcb23048>
効果|<google.cloud.language.syntax.PartOfSpeech object at 0x7f2efcb236a0>
、|<google.cloud.language.syntax.PartOfSpeech object at 0x7f2efcb23748>
2030|<google.cloud.language.syntax.PartOfSpeech object at 0x7f2efcb23710>
年|<google.cloud.language.syntax.PartOfSpeech object at 0x7f2efcb237f0>
まで|<google.cloud.language.syntax.PartOfSpeech object at 0x7f2efcb23828>
に|<google.cloud.language.syntax.PartOfSpeech object at 0x7f2efcb23908>
1780|<google.cloud.language.syntax.PartOfSpeech object at 0x7f2efcbb1da0>
兆|<google.cloud.language.syntax.PartOfSpeech object at 0x7f2efcbc1e80>
円|<google.cloud.language.syntax.PartOfSpeech object at 0x7f2efcbc1eb8>
規模|<google.cloud.language.syntax.PartOfSpeech object at 0x7f2efcb23940>


In [9]:
token = tokens[0]
dir(token.part_of_speech)

['ADJECTIVE',
 'ADPOSITION',
 'ADVERB',
 'AFFIX',
 'CARDINAL_NUMBER',
 'CONJUNCTION',
 'DETERMINER',
 'NOUN',
 'OTHER',
 'PARTICIPLE',
 'PRONOUN',
 'PUNCTUATION',
 'UNKNOWN',
 'VERB',
 '_REVERSE_MAP',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'aspect',
 'case',
 'form',
 'from_api_repr',
 'gender',
 'mood',
 'number',
 'person',
 'proper',
 'reciprocity',
 'reverse',
 'tag',
 'tense',
 'voice']

In [5]:
# https://cloud.google.com/storage/docs/reference/libraries
from google.cloud import storage

storage_client = storage.Client()

bucket = storage_client.get_bucket('qa-nlp-1')

print('Bucket: {}.'.format(bucket.name))

NotFound: 404 Not Found (GET https://www.googleapis.com/storage/v1/b/qa-nlp-1?projection=noAcl)